***Stock Price Chart***


Instalacja wymaganych bibliotek:

In [ ]:
!pip install plotly requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Uzyskanie przez program dostępu do Google Drive w celu pobrania z pliku tekstowego klucza API. Klucz można wygenerować na stronie https://www.alphavantage.co/support/#support

In [18]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Ścieżka do folderu w którym znajduje się API key powinna zostać zaktualizowana przez użytkownika tak aby odpowiadała folderowi w którym znajduje się plik tekstowy z kluczem do API.

In [19]:
%cd /content/gdrive/My Drive/Projekt

/content/gdrive/My Drive/Projekt


In [20]:
!ls

api_key.txt


Importowanie wymaganych bilbliotek:

In [21]:
import requests
import plotly.graph_objects as go
import re
import numpy as np

GET_STOCK_PRICES - przekazuje zdefiniowane symbole akcji, klucz API, wybraną funkcję oraz ilość wybranych przez użytkownika dni. 

In [22]:
def get_stock_prices(symbols, api_key, num_days):
    base_url = "https://www.alphavantage.co/query"
    params = {
        "function": "TIME_SERIES_DAILY_ADJUSTED",
        "apikey": api_key
    }

    price_data = {}
    for symbol in symbols:
        params['symbol'] = symbol
        response = requests.get(base_url, params=params)
        data = response.json()

        if 'Time Series (Daily)' in data:
            price_data[symbol] = extract_price_data(data, num_days)
        else:
            price_data[symbol] = None

    return price_data

EXTRACT_PRICE_DATA - wyciąga kursy na giełdzie dla wybranej ilości dni.

In [23]:
def extract_price_data(data, num_days):
    price_data = data['Time Series (Daily)']
    dates = list(price_data.keys())[:num_days]
    prices = [float(price_data[date]['4. close']) for date in dates]
    return dates, prices

Po wpisaniu symboli akcji wybranych spółek giełdowych oraz ilości dni następuje sprawdzenie czy podane wartości spełniają kryteria - dla symboli ważne jest by składały się one z samych liter, natomiast ilość dni musi być wartością dodatnią. 

Jeśli podany symbol akcji znajduje się w bazie to zostanie on umieszczony na wykresie, jeśli nie, to zostanie on przez program pominięty.

Wykres może służyć do porównania kilku wykresów cen akcji lub służyć jako podgląd aktualnych wartości wybranej akcji.

In [24]:
def main():
    with open('api_key.txt', 'r') as file:
      api_key = file.read().strip()


    while True:
        symbols_input = input("Enter the stock symbols (separated by commas): ")
        symbols = [symbol.strip() for symbol in symbols_input.split(',')]
        if all(re.match(r'^[a-zA-Z0-9\.\-\^]+$', symbol) for symbol in symbols):
            break
        else:
            print("Invalid stock symbols. Please try again.")


    while True:
        try:
            num_days = int(input("Enter the number of days to include in the chart: "))
            if num_days > 0:
                break
            else:
                print("Number of days must be a positive integer.")
        except ValueError:
            print("Invalid input. Please enter a valid number.")

    price_data = get_stock_prices(symbols, api_key, num_days)

    fig = go.Figure()

    for symbol in symbols:
        if price_data[symbol]:
            dates, prices = price_data[symbol]
            fig.add_trace(go.Scatter(x=dates, y=prices, name=symbol))

            x_values = np.array(range(len(dates)))
            coefficients = np.polyfit(x_values, prices, 1)
            trendline = np.polyval(coefficients, x_values)
            fig.add_trace(go.Scatter(x=dates, y=trendline, name=f"{symbol} Trend"))

    fig.update_layout(
        title="Stock Price Chart",
        xaxis_title="Date",
        yaxis_title="Price",
        xaxis=dict(tickangle=45),
        legend=dict(x=0.01, y=0.99)
        
    )

    fig.show()

main()

Enter the stock symbols (separated by commas): AAPL, AMD, AMZN
Enter the number of days to include in the chart: 50
